In [138]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.linear_model import RidgeClassifier

In [139]:
import neptune

neptune.init(project_qualified_name='', # change this to your `workspace_name/project_name`
             api_token='', # change this to your api token
            )

Project(vovandro/alfabattle1)

In [140]:
df = pd.read_csv('alfabattle2_abattle_train_target.csv', parse_dates=['timestamp'])

In [141]:
df.head()

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen


In [143]:
class_target = {'main_screen':0, 'invest':1, 'statement':2, 'phone_money_transfer':3,
       'own_transfer':4, 'credit_info':5, 'chat':6, 'card2card_transfer':7,
       'mobile_recharge':8, 'card_recharge':9,
        0:'main_screen', 1:'invest', 2:'statement', 3:'phone_money_transfer',
       4:'own_transfer', 5:'credit_info', 6:'chat', 7:'card2card_transfer',
       8:'mobile_recharge', 9:'card_recharge'}

In [144]:
#df['main_screen'] = df['multi_class_target'] == 'main_screen'
#df['invest'] = df['multi_class_target'] == 'invest'
#df['statement'] = df['multi_class_target'] == 'statement'
#df['phone_money_transfer'] = df['multi_class_target'] == 'phone_money_transfer'
#df['own_transfer'] = df['multi_class_target'] == 'own_transfer'
#df['credit_info'] = df['multi_class_target'] == 'credit_info'
#df['chat'] = df['multi_class_target'] == 'chat'
#df['card2card_transfer'] = df['multi_class_target'] == 'card2card_transfer'
#df['mobile_recharge'] = df['multi_class_target'] == 'mobile_recharge'
#df['card_recharge'] = df['multi_class_target'] == 'card_recharge'
df['class_target'] = df['multi_class_target'].apply(lambda row: class_target[row]).astype(np.int8)

In [145]:
df['y'] = df.timestamp.dt.year
df['m'] = df.timestamp.dt.month
df['d'] = df.timestamp.dt.day
df['dw'] = df.timestamp.dt.dayofweek
df['h'] = df.timestamp.dt.hour

In [146]:
del df['timestamp']
del df['multi_class_target']
del df['session_id']

In [147]:
df[df['client_pin'] == 	'f0c674b2bb4dc64be607029271d706ec']

,client_pin,class_target,y,m,d,dw,h
695067,f0c674b2bb4dc64be607029271d706ec,9,2020,7,5,6,7
1409788,f0c674b2bb4dc64be607029271d706ec,5,2020,7,28,1,19
2535361,f0c674b2bb4dc64be607029271d706ec,5,2020,7,12,6,11
4707602,f0c674b2bb4dc64be607029271d706ec,9,2020,7,14,1,17


In [148]:
df_pred = pd.read_csv('alfabattle2_prediction_session_timestamp.csv', parse_dates=['timestamp'])
df_pred['y'] = df_pred.timestamp.dt.year
df_pred['m'] = df_pred.timestamp.dt.month
df_pred['d'] = df_pred.timestamp.dt.day
df_pred['dw'] = df_pred.timestamp.dt.dayofweek
df_pred['h'] = df_pred.timestamp.dt.hour
del df_pred['timestamp']

In [ ]:
neptune.create_experiment(name='in_user_model', upload_source_files=['main.ipynb'])

In [152]:
result = []
for client in df_pred.iterrows():
    # class_target != main_screen
    X = df[(df['client_pin'] == client[1]['client_pin']) & (df['class_target'] != 0)]
    y = X['class_target']
    del X['client_pin']
    del X['class_target']
    X_test = client[1].copy()
    del X_test['client_pin']
    
    if X.shape[0] != 0:
        model = RidgeClassifier(random_state=6).fit(X, y)
        y_pred = model.predict([X_test])
    else:
        y_pred = [2]
    
    result.append([client[1]['client_pin'], class_target[y_pred[0]]])

In [153]:
#neptune.log_artifact('model.pkl')
df_result = pd.DataFrame(result, columns=['client_pin', 'prediction'])
df_result.to_csv('predict_ridge_individual_no_ms.csv', index=False)

In [154]:
neptune.stop()

In [124]:
df_result

,client_pin,prediction
0,f0c674b2bb4dc64be607029271d706ec,card_recharge
1,90725b54ce77576883813d87749df6bd,main_screen
2,eb0f82d74c7b7bd5eafbd5b5f8cb3e2a,statement
3,831bf4c0ecccc55e536b8cfb9153d672,statement
4,3f1a5a1c492ce877af833113e59c5797,main_screen
...,...,...
79263,4e46e206d38dcb7af41d9ced20d50ac2,statement
79264,5b59bf5bd1e07775b1bc95dd7151d2cd,main_screen
79265,94bcc852dd7f806f6c733056d1b0be7b,statement
79266,2061f779928217eea2878fd3b605fafd,main_screen
